# Assignment 05: Mustaeen Ahmed

In [16]:
import pygame
import moderngl
import numpy
from math import cos, sin, radians

In [17]:
pygame.init()
screen_info = pygame.display.Info()

WIDTH = screen_info.current_w
HEIGHT = screen_info.current_h

clock = pygame.time.Clock()
running = True

pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MAJOR_VERSION, 3)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MINOR_VERSION, 3)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_FORWARD_COMPATIBLE_FLAG, True)

In [18]:
pygame.display.set_mode((WIDTH, HEIGHT), flags=pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)
pygame.display.set_caption(title="Assignment 04: Mustaeen Ahmed")
gl = moderngl.get_context()

texture_img = pygame.image.load("clockFace.png")
texture_data = pygame.image.tobytes(texture_img, "RGB")
faceTexture = gl.texture(texture_img.get_size(), data=texture_data, components=3)
face_sampler = gl.sampler(texture=faceTexture, filter=(gl.LINEAR, gl.LINEAR))

In [19]:
position_data = [
    0.0, 0.8,
    0.8, 0.0,
    -0.8, 0.0,

    0.0, -0.8,
    0.8, 0.0,
    -0.8, 0.0
]

triangle_vertex_positions = numpy.array(position_data).astype("float32")
triangle_position_buffer = gl.buffer(triangle_vertex_positions)

In [20]:
fragment_shader_code = '''
    #version 330 core
    
    out vec4 color;
    uniform vec3 inColor;

    void main() {
        color = vec4(inColor, 0);
    }
'''

vertex_shader_code = '''
    #version 330 core

    layout (location = 0) in vec2 position;
    uniform float scale;
    uniform float distance;
    uniform mat2 M;

    void main() {
        vec2 d_vector = distance * vec2(1, 0);
        vec2 P = position * scale + d_vector;
        gl_Position = vec4(M * P, 0.0, 1.0);
    }
'''

vertex_shader_code_line = '''
    #version 330 core

    uniform float distance;
    uniform mat2 M;
    
    void main() {
        vec2 position = vec2(0);
        if (gl_VertexID > 0) {
            vec2 d_vector = distance * vec2(1, 0);
            position += d_vector;
        }
        gl_Position = vec4(M * position, 0.0, 1.0);
    }
'''

vertex_shader_code_face = '''
    #version 330 core

    out vec2 f_uv;
    void main() {
        vec2 P, uv;
    
        switch (gl_VertexID) {
            case 0:
                uv = vec2(0, 0);
                P = vec2(-1, -1);
                break;
            case 1:
                uv = vec2(1, 0);
                P = vec2(1, -1);
                break;
            case 2:
            case 4:
                uv = vec2(1, 1);
                P = vec2(1, 1);
                break;
            case 5:
                uv = vec2(0, 1);
                P = vec2(-1, 1);
                break;
        }
        gl_Position = vec4(P, 0.0, 1.0);
    }
'''

fragment_shader_code_face = '''
    #version 330 core
    in vec2 f_uv;

    out vec4 color;
    uniform sampler2D map;

    void main() {
        color = texture(map, f_uv);
    }
'''

In [21]:
program = gl.program(
    vertex_shader = vertex_shader_code,
    fragment_shader = fragment_shader_code
)

program_line = gl.program(
    vertex_shader = vertex_shader_code_line,
    fragment_shader= fragment_shader_code
)

program_face = gl.program(
    vertex_shader=vertex_shader_code_face,
    fragment_shader=fragment_shader_code_face,
)

renderable = gl.vertex_array(program,
    [( triangle_position_buffer, "2f", "position" )])

line_renderable = gl.vertex_array(program_line, [])

Error: GLSL Linker failed

Program
=======
ERROR: Input of fragment shader 'f_uv' not written by vertex shader



In [ ]:
angle = 270

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == 27):
            running = False
        elif event.type == pygame.VIDEORESIZE:
            WIDTH, HEIGHT = event.size
            aspect_ratio = WIDTH / HEIGHT

    gl.clear(0.5, 0.5, 0)

    aspect_ratio = WIDTH / HEIGHT
    angle_r = radians(-angle)

    M = numpy.array([
        [cos(angle_r) / aspect_ratio, -sin(angle_r) / aspect_ratio],
        [sin(angle_r), cos(angle_r)]
    ], dtype='f4')

    program['M'].write(M.T.tobytes())
    program_line['M'].write(M.T.tobytes())

    program['scale'] = 0.1
    program['distance'] = 0
    program['inColor'] = [1, 1, 0]

    renderable.render(moderngl.TRIANGLES, vertices=6)

    program['distance'] = 0.8
    renderable.render(moderngl.TRIANGLES, vertices=6)

    program_line['distance'] = 0.8
    line_renderable.render(moderngl.LINES, vertices=2)

    pygame.display.flip()
    clock.tick(60)
    angle = angle + 0.1
    if angle > 360:
        angle = 0

pygame.quit()